In [5]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

#---------------------(Read Dataset)-------------------------------------------
df=pd.read_csv("C:/Users/Asus/Desktop/Forecasting-covidcases/dataset-fr.csv")
df['date']=pd.to_datetime(df['date']).dt.date
df.set_index('date',inplace=True)
#------------------(Split Dataset)---------------------------------------------
TRAIN_SPLIT=int(len(df)*0.8)

lag=21 #for active cases (conf_j1)
#------------------(convert to supervised)-------------------------------------
#------------------(Multivariate_data function)--------------------------------
def multivariate_data(dataset, target, start_index, end_index, history_size,
                      target_size, step, single_step=False):
    data = []
    labels = []

    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size

    for i in range(start_index, end_index):
        indices = range(i-history_size, i, step)
        data.append(dataset[indices])

        if single_step:
            labels.append(target[i+target_size])
        else:
            labels.append(target[i:i+target_size])

    return np.array(data), np.array(labels)
#------------------------(set variable and convert to supervised)--------------
#the function just accept the array type
df=np.array(df)
#----------------------------(Scale Data)--------------------------------------
scaler =MinMaxScaler(feature_range=(0,1))
df=scaler.fit_transform(df)
past_history =lag
#number of step for prediction
future_target = 1
STEP = 1

x_train_multi, y_train_multi= multivariate_data(df, df[:,12], 0,
                                                   TRAIN_SPLIT, past_history,
                                                   future_target, STEP,
                                                   single_step=False)
x_test_multi, y_test_multi = multivariate_data(df, df[:,12],
                                               TRAIN_SPLIT, None, past_history,
                                               future_target, STEP,
                                               single_step=False)

In [6]:
print(x_train_multi.shape ,x_test_multi.shape ,y_train_multi.shape,y_test_multi.shape)

x_train_2d=x_train_multi.reshape(x_train_multi.shape[0],(x_train_multi.shape[1]*x_train_multi.shape[2]))
x_test_2d=x_test_multi.reshape(x_test_multi.shape[0],-1)                                
y_train_1d=y_train_multi.ravel()
y_test_1d=y_test_multi.ravel()
           
print("********after conver***********")              
print(x_train_2d.shape ,x_test_2d.shape ,y_train_1d.shape,y_test_1d.shape)
#------------------(define models and fit)-------------------------------------
#---------------------(LR)-----------------------------------------------------
LR_model=LinearRegression()
LR_model.fit(x_train_2d,y_train_1d)

#------------------(MSE and Predict)-------------------------------------------
##Predict Test Set
predict_LR=LR_model.predict(x_test_2d)
mse_LR= mean_squared_error(y_test_1d, predict_LR)
print("MSE_LR : ",  mse_LR)
##-------------------(predict next step)---------------------------------------
input_samples=x_test_2d[-1:,:]
next_step_LR=LR_model.predict(input_samples)
#print(next_step_RNN.shape)
print("Next step LR :" , next_step_LR)

(515, 21, 15) (112, 21, 15) (515, 1) (112, 1)
********after conver***********
(515, 315) (112, 315) (515,) (112,)
MSE_LR :  0.0059991756514626
Next step LR : [0.8431092]


In [7]:

#Building the support vector regression model
#we will choose the best parameter for SVR model
"""
kernel = ['poly','sigmoid','rbf']
C = [0.01,0.1,1,10]
gamma = [0.01,0.1,1]
epsilon = [0.01,0.1,1]
shrinking = [True,False]
svr_grid = {'kernel':kernel,'C':C,'gamma':gamma,'epsilon':epsilon,'shrinking':shrinking}
"""
SVR = SVR(C=1, epsilon=0.01, gamma=0.01, kernel='poly')
#svr_search = RandomizedSearchCV(SVR,svr_grid,cv=3)
SVR.fit(x_train_2d,y_train_1d)
#svr_search.best_params_
#svr_confirmed=svr_search.best_estimator_
#svr_confirmed.fit(x_train_2d,y_train_1d)
#------------------(MSE and Predict)------------------------------------------
prediction_SVR = SVR.predict(x_test_2d)
mse_SVR = mean_squared_error(y_test_1d,prediction_SVR)
print("MSE_SVR:" , mse_SVR)
##-------------------(predict next step)---------------------------------------
input_samples=x_test_2d[-1:,:]
next_step_SVR=SVR.predict(input_samples)
print("Next step SVR :",next_step_SVR)

MSE_SVR: 0.035253512238210334
Next step SVR : [0.42545896]


In [8]:
#------------------------------------(Random Forest)---------------------------

RF = RandomForestRegressor(max_depth=80, min_samples_leaf=4, min_samples_split=10,
                      n_estimators=200)

RF.fit(x_train_2d,y_train_1d)
#print(RF_confirmed)
#------------------(MSE and Predict)-------------------------------------------
prediction_RF= RF.predict(x_test_2d)
mse_RF = mean_squared_error(y_test_1d,prediction_RF)
print("MSE_RF:" , mse_RF)
##-------------------(predict next step)---------------------------------------
input_samples=x_test_2d[-1:,:]
next_step_RF=RF.predict(input_samples)
print("Next step RF :",next_step_RF)

MSE_RF: 0.06791176453470113
Next step RF : [0.0988711]
